#### Loading model

In [ ]:
from pymgpipe import *

# run through workflow.ipynb first if this file doesn't exist
model = load_model('out/problems/mc1.mps.7z')

#### Fetching reactions

`get_reactions` returns a list of `optlang.Variable` objects, that all have a name (id), lower bound, and upper bound

You can fetch reactions either by passing in an explicit list of reaction IDs using the `reactions` param, or by using a regex match using the `regex` param.

`get_reactions(model)` without any parameters will get all exchange reactions by default. 

Remember- this will only get the **forward** reaction

In [ ]:
# Getting ALL reactions
all_reactions = get_reactions(model, regex='.*')

# Getting all reactions with the word biomass in ID
biomass_reactions = get_reactions(model,regex='.*biomass.*')

# Getting ALL exchange reactions
exchange_reactions = get_reactions(model, regex = Constants.EX_REGEX)

# Getting specific list of reactions
specific_reaction = get_reactions(model, reactions=['EX_tyr_L[fe]','EX_ala_L[fe]','Diet_EX_glu_L[d]'])

for r in specific_reaction:
    print(r.name, r.lb, r.ub)

# You can also get reactions by passing in a list of optlang.Variable's 
first_five_reactions = get_reactions(model, reactions=model.variables[:5])

# You can also get individual reactions like this
ind = model.variables['EX_tyr_L[fe]'] # by name
another_ind = model.variables[100] # by index

# You can check to see if a certain variable exists within the model like so
exists = 'EX_tyr_L[fe]' in model.variables # <-- TRUE
not_exists = 'bad_reaction' in model.variables  # <-- FALSE

#### Getting reverse reaction

If you want to calculate **NET flux** for a reaction, you will need to get the **reverse** reactions as well

In [ ]:
exchange_reactions = get_reactions(model, regex = Constants.EX_REGEX)
for e in exchange_reactions:
    reverse_var = get_reverse_var(model, e) # this returns the actual reverse optlang.Variable
    reverse_id = get_reverse_id(e.name) # if you just want the id of the reverse reaction, you can do it like so
    print(e.name,reverse_id)

Now, a quick way to calculate the NET value for all exchanges would be done like so-

(I will add a function soon where you can just provide a reaction name and it returns the net flux expression)

In [7]:
net_exchanges = [forward - get_reverse_var(model,forward) for forward in exchange_reactions]

# or alternatively, like this (produces the same results)
net_exchanges = [forward - model.variables[get_reverse_id(forward.name)] for forward in exchange_reactions]

### Constraining reactions

To constrain reactions you essentially are just setting the lower and upper bound, like so

In [ ]:
tyr_v = model.variables['EX_tyr_L[fe]']
print('Before-',(tyr_v.lb,tyr_v.ub))

# lets say you want to fix tyramine to equal 500
tyr_v.set_bounds(500,500)
print('After-',(tyr_v.lb,tyr_v.ub))

BUT! Notice in the example above, we are only fixing the **forward** reaction

In order to easily fix the **net** value for a certain reaction/metabolite, I made a function that takes in a map of `{id: value}`

Any reactions in the map that don't exist within the model will be ignored

In [ ]:
reactions_to_fix = {
    'Diet_EX_tyr_L[d]': 25,
    'Diet_EX_ala_L[d]': 50,
    'Diet_EX_glu_L[d]': 100,
    'bad_rxn': 50,
}

# try changing the threshold to see how the output changes
constrain_reactions(model,flux_map=reactions_to_fix,threshold=0.0)

for m in reactions_to_fix:
    if m in model.variables: # try removing this IF statement and see what happens
        v = model.variables[m]
        print(v.name, v.lb, v.ub)

### Setting objectives and solving

Let's say you want to optimize for sum of all exchanges-

In [ ]:
model = load_model('out/problems/mc1.mps.7z')

exchange_reactions = get_reactions(model,regex='EX_.*[fe]')
net_exchanges = [f - get_reverse_var(model, f) for f in exchange_reactions]

objective_expr = np.sum(net_exchanges)
objective_expr

Now you set the objective and solve!

`solve_model` will return the NET flux solution for all reactions, so no need to worry about that

In [ ]:
set_objective(model, objective_expr, direction='max')
sol = solve_model(model, ex_only=True) # if you want solution for only exchanges
sol = solve_model(model, ex_only=False) # if you want solution for ALL reactions
sol = solve_model(model, reactions=['Diet_EX_glu_L[d]','Diet_EX_ala_L[d]','communityBiomass']) # if you want solution for only a subset of reactions
sol

The solution is a dataframe, so you can filter just like you would with any pandas dataframe

In [28]:
sol.loc[sol.index.str.contains('communityBiomass')] # sums up to 1 like it's supposed to!

,mc1
communityBiomass,1.0


#### Running FVA

In [ ]:
# This is basically how you would run it, I'd check the method signature to see other params and default values
sol = regularFVA(
    model = model,
    reactions = get_reactions(model,regex='EX_.*[fe]')[:10],
    parallel = True,
    threads = 5
)
sol